In [1]:
import pandas as pd

In [2]:
# Carregando os dados e ajustando o datetime
tsla_data = pd.read_csv("data/TSLA.csv")
tsla_data["Date"] = pd.to_datetime(tsla_data["Date"])

tsla_data = pd.Series(tsla_data["Adj Close"].values, index = tsla_data["Date"])
tsla_data

Date
2010-06-29      1.592667
2010-06-30      1.588667
2010-07-01      1.464000
2010-07-02      1.280000
2010-07-06      1.074000
                 ...    
2024-08-15    214.139999
2024-08-16    216.119995
2024-08-19    222.720001
2024-08-20    221.100006
2024-08-21    223.270004
Length: 3561, dtype: float64

In [8]:
for i, j in zip(tsla_data.index, tsla_data.values):
    print(i, j)

2010-06-29 00:00:00 1.592667
2010-06-30 00:00:00 1.588667
2010-07-01 00:00:00 1.464
2010-07-02 00:00:00 1.28
2010-07-06 00:00:00 1.074
2010-07-07 00:00:00 1.053333
2010-07-08 00:00:00 1.164
2010-07-09 00:00:00 1.16
2010-07-12 00:00:00 1.136667
2010-07-13 00:00:00 1.209333
2010-07-14 00:00:00 1.322667
2010-07-15 00:00:00 1.326
2010-07-16 00:00:00 1.376
2010-07-19 00:00:00 1.460667
2010-07-20 00:00:00 1.353333
2010-07-21 00:00:00 1.348
2010-07-22 00:00:00 1.4
2010-07-23 00:00:00 1.419333
2010-07-26 00:00:00 1.396667
2010-07-27 00:00:00 1.37
2010-07-28 00:00:00 1.381333
2010-07-29 00:00:00 1.356667
2010-07-30 00:00:00 1.329333
2010-08-02 00:00:00 1.394667
2010-08-03 00:00:00 1.463333
2010-08-04 00:00:00 1.417333
2010-08-05 00:00:00 1.363333
2010-08-06 00:00:00 1.306
2010-08-09 00:00:00 1.306667
2010-08-10 00:00:00 1.268667
2010-08-11 00:00:00 1.193333
2010-08-12 00:00:00 1.173333
2010-08-13 00:00:00 1.221333
2010-08-16 00:00:00 1.252
2010-08-17 00:00:00 1.276667
2010-08-18 00:00:00 1.2513

In [20]:
class Backtest():
    def __init__(self, stock: pd.Series, function, cash: int, time_start: str, time_stop: str):
        self.stock = stock
        self.function = function
        self.cash = cash
        self.time_start = time_start
        self.time_stop = time_stop
    
    def stats(self):
        # Filtrando os dados para o período selecionado
        filtered_data = self.stock[(self.stock.index >= pd.to_datetime(self.time_start)) & (self.stock.index <= pd.to_datetime(self.time_stop))]
        
        current_state = 0

        for date, price in zip(filtered_data.index, filtered_data.values):
            if self.function[date] == 1:
                if current_state == 1:
                    continue
                elif current_state == 0:
                    self.cash -= price
                else:
                    self.cash -= 2*price

                current_state = 1

            elif self.function[date] == 0:
                if current_state == 1:
                    self.cash += price
                elif current_state == 0:
                    continue
                else:
                    self.cash -= price

                current_state = 0

            else:
                if current_state == 1:
                    self.cash += 2*price
                elif current_state == 0:
                    self.cash += price
                else:
                    continue

                current_state = -1

        return self.cash

In [14]:
import numpy as np

# Classe que servirá como classe abstrata para as estratégias desenvolvidas.
class Strategy():
    def __init__(self) -> None:
        pass

    def run(self):
        pass

    # Função para calcular a média móvel exponencial dos dados
    def moving_average(self, data: pd.Series, size: int) -> pd.Series:
        average = data.ewm(span = size, adjust = False).mean()
        return average


class MA_crossover(Strategy):
    def __init__(self, data: pd.Series, short_period: int, long_period: int) -> None:
        super().__init__()
        self.data = data
        self.short_period = short_period
        self.long_period = long_period
    
    def run(self):
        long_average = self.moving_average(self.data, self.long_period)
        short_average = self.moving_average(self.data, self.short_period)
        
        # Fazendo a interseção dos valores das médias de acordo com as datas
        long_average_aligned, short_average_aligned = long_average.align(short_average, join = "inner")
        # Calculando a diferença entre as médias
        diff = short_average_aligned - long_average_aligned
        # Calculando a variação diária da média curta
        short_average_var = short_average_aligned.diff()
        
        # Se a média curta está maior que a longa e está subindo, compra
        diff[(diff * short_average_var > 0) & (diff > 0)] = 1
        # Se a média curta está menor que a longa e está descendo, vende
        diff[(diff * short_average_var > 0) & (diff < 0)] = -1
        # Se a média curta está menor que a longa e está subindo ou maior e descendo, fica neutro
        diff[diff * short_average_var < 0] = 0

        return diff

In [22]:
teste = MA_crossover(tsla_data, 8, 21)

actions = teste.run()

backtest = Backtest(tsla_data, actions, 1000, "2022-01-01", "2023-01-01")

backtest.stats()

1245.8999319999998